# Storing Data in BigQuery

Before running any code in this Jupyter Notebook, please ensure you have the following Python libraries installed:
- **Pandas**: This library is a powerful tool for data manipulation and analysis.
- **Google OAuth2**: This library is used for Google Cloud authentication.
- **Cryptography**: This library provides tools for encryption and decryption.

If you haven't installed these libraries yet, you can do so using the following commands in a code cell:
- `!pip install pandas`
- `!pip install google-auth`
- `!pip install cryptography`

In [1]:
import pandas as pd

from google.oauth2.service_account import Credentials
from cryptography.fernet import Fernet

pd.set_option("display.max_columns", None)

Before running the code to load the data, please ensure that the data you want to load is in CSV format. Specifically, for the daily data, make sure the CSV file contains the following columns:

`['AREA', 'DATE_ID', 'ERBS', 'EUTRANCELLFDD', 'LTE_CSFB_SR', 'RRC Setup Success Rate (Service)', 'ERAB_Setup_Success_Rate_All', 'Session_Setup_Success_Rate', 'Session_Abnormal_Release', 'Intra-Frequency Handover Out Success Rate', 'Radio_Network_Availability_Rate', 'Cell_Downlink_Average_Throughput', 'Cell_Uplink_Average_Throughput', 'DL_PDCP_User_Throughput', 'User_Uplink_Average_Throughput', 'DL_Resource_Block_Utilizing_Rate', 'UL_Resource_Block_Utilizing_Rate', 'Downlink_Traffic_Volume', 'Uplink_Traffic_Volume', 'Total_Traffic_Volume', 'Maximum_User_Number_RrcConn', 'pm_count', 'Max DL Cell Downlink Throughput', 'Max UL Cell Downlink Throughput', 'inter_freq_HO', 'UL_RSSI_dbm', 'SE_2', 'Average_CQI_nonHOME', 'volte_traffic_erl', 'volte_payload_DL', 'volte_payload_UL', 'pmCellDownTimeAuto', 'pmCellDownTimeMan', 'L_LATENCY_DL_ms', 'Integrity_PacketLossRate_DL', 'Integrity_PacketLossRate_UL']`

For the hourly data, ensure that the CSV file contains the following columns:

`['AREA', 'DATE_ID', 'HOUR_ID', 'ERBS', 'EUTRANCELLFDD', 'LTE_CSFB_SR', 'RRC Setup Success Rate (Service)', 'ERAB_Setup_Success_Rate_All', 'Session_Setup_Success_Rate', 'Session_Abnormal_Release', 'Intra-Frequency Handover Out Success Rate', 'Radio_Network_Availability_Rate', 'Cell_Downlink_Average_Throughput', 'Cell_Uplink_Average_Throughput', 'DL_PDCP_User_Throughput', 'User_Uplink_Average_Throughput', 'DL_Resource_Block_Utilizing_Rate', 'UL_Resource_Block_Utilizing_Rate', 'Downlink_Traffic_Volume', 'Uplink_Traffic_Volume', 'Total_Traffic_Volume', 'Max DL Cell Downlink Throughput', 'Max UL Cell Downlink Throughput', 'L_LATENCY_DL_ms', 'Integrity_PacketLossRate_DL', 'Integrity_PacketLossRate_UL', 'inter_freq_HO', 'UL_RSSI_dbm', 'SE_2', 'Average_CQI_nonHOME', 'volte_traffic_erl', 'volte_payload_DL', 'volte_payload_UL', 'Band_', 'Sector_', 'NEID_', 'SiteID_']`

Please adjust the file name in the code to match your actual data CSV file.

In [2]:
df = pd.read_csv("4g_daily.csv")
df.head()

,AREA,DATE_ID,ERBS,EUTRANCELLFDD,LTE_CSFB_SR,RRC Setup Success Rate (Service),ERAB_Setup_Success_Rate_All,Session_Setup_Success_Rate,Session_Abnormal_Release,Intra-Frequency Handover Out Success Rate,Radio_Network_Availability_Rate,Cell_Downlink_Average_Throughput,Cell_Uplink_Average_Throughput,DL_PDCP_User_Throughput,User_Uplink_Average_Throughput,DL_Resource_Block_Utilizing_Rate,UL_Resource_Block_Utilizing_Rate,Downlink_Traffic_Volume,Uplink_Traffic_Volume,Total_Traffic_Volume,Maximum_User_Number_RrcConn,pm_count,Max DL Cell Downlink Throughput,Max UL Cell Downlink Throughput,inter_freq_HO,UL_RSSI_dbm,SE_2,Average_CQI_nonHOME,volte_traffic_erl,volte_payload_DL,volte_payload_UL,pmCellDownTimeAuto,pmCellDownTimeMan,L_LATENCY_DL_ms,Integrity_PacketLossRate_DL,Integrity_PacketLossRate_UL
0,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002ML1,0.0,99.99393,99.98195,99.97589,0.06679,99.66047,99.97338,19789.33210,2677.57289,36897.11111,8292.20991,17.656,10.541,56468.78425,5908.32513,62377.10938,51,96,147057,40975,99.54023,-117.74849,2.59416,11.543,0.54444,0.00426,0.00255,23,0,3.14533,0.00322,0.01026
1,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002ML2,0.0,99.88980,99.91135,99.80125,0.09515,99.41480,99.98380,18049.37516,1739.05075,20333.29218,1032.97104,33.239,17.802,97340.72600,9219.08388,106559.80987,83,96,177330,31804,98.71007,-113.52638,1.88985,10.274,1.47361,0.01188,0.00950,14,0,3.54175,0.00866,0.07735
2,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002ML3,0.0,99.97177,99.93837,99.91015,0.06221,99.11193,99.98148,20524.85892,2059.30252,30578.76421,1609.04242,18.405,10.784,48805.99038,4460.34025,53266.33063,41,96,151519,47438,98.73418,-114.56841,2.11394,10.925,0.18611,0.00181,0.00121,16,0,3.38281,0.00714,0.37989
3,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002MT1,0.0,99.57377,99.46474,99.04079,0.24718,99.87531,99.98727,11494.63908,2067.32164,16281.88406,3348.21321,19.14,11.401,24817.88875,3556.79400,28374.68275,24,96,107227,25911,98.60075,-119.60126,2.04409,11.349,0.34167,0.00284,0.00172,11,0,3.44817,0.03223,0.01450
4,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002MT2,0.0,99.24931,99.09149,98.34761,0.44175,99.49495,99.98843,10570.76424,1568.23771,10733.56481,1379.71641,31.603,11.964,39139.74400,4135.48675,43275.23075,31,96,57049,15136,98.67769,-113.3184,1.60831,10.997,0.18333,0.00147,0.00126,10,0,3.87995,0.02036,0.02991


In [3]:
df_hourly = pd.read_csv("4g_hourly_dummy.csv")
df_hourly.head()

,AREA,DATE_ID,HOUR_ID,ERBS,EUTRANCELLFDD,LTE_CSFB_SR,RRC Setup Success Rate (Service),ERAB_Setup_Success_Rate_All,Session_Setup_Success_Rate,Session_Abnormal_Release,Intra-Frequency Handover Out Success Rate,Radio_Network_Availability_Rate,Cell_Downlink_Average_Throughput,Cell_Uplink_Average_Throughput,DL_PDCP_User_Throughput,User_Uplink_Average_Throughput,DL_Resource_Block_Utilizing_Rate,UL_Resource_Block_Utilizing_Rate,Downlink_Traffic_Volume,Uplink_Traffic_Volume,Total_Traffic_Volume,Max DL Cell Downlink Throughput,Max UL Cell Downlink Throughput,L_LATENCY_DL_ms,Integrity_PacketLossRate_DL,Integrity_PacketLossRate_UL,inter_freq_HO,UL_RSSI_dbm,SE_2,Average_CQI_nonHOME,volte_traffic_erl,volte_payload_DL,volte_payload_UL,Band_,Sector_,NEID_,SiteID_
0,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108ME1,0,100.0000,100.00000,100.00000,0.0,100.0,100.0,18.452752,1.027181,36.291791,0.500244,13.833,15,1.526871,0.097257,1.624129,57.013,3.057,4.45460,0.00244,0.06323,100,-111.24809,2.58517,11.558,0,0,0,L2300,1,SAA108ME1,SAA108
1,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108ME2,0,100.0000,100.00000,100.00000,0.0,100.0,100.0,17.245951,1.687589,44.481812,1.115079,18.833,22.667,3.489945,0.263139,3.753084,83.282,3.058,2.78735,0.00270,0.06924,100,-113.47277,3.8755,12.635,0,0,0,L2300,2,SAA108ME1,SAA108
2,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108ME3,0,0.0000,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,\N,\N,0.000000,0.000000,0.000000,0.000,0.000,0.00000,0.00000,0.00000,\N,\N,\N,\N,0,0,0,L2300,3,SAA108ME1,SAA108
3,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108MF1,0,99.9239,99.87849,99.80248,0.0,0.0,100.0,19.568129,0.632613,45.233267,0.285659,12.833,10.333,1.518476,0.039733,1.558209,57.273,1.005,5.45246,0.12481,1.84608,100,-109.93618,3.10492,12.473,0,0,0,L2300,1,SAA108ME1,SAA108
4,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108MF2,0,100.0000,100.00000,100.00000,0.0,100.0,100.0,17.565332,1.701387,29.490436,1.012616,26,22,3.706544,0.251886,3.958430,70.168,2.405,3.44076,0.00505,0.00054,100,-113.14835,2.75921,12.306,0,0,0,L2300,2,SAA108ME1,SAA108


Before storing the data in the BigQuery database, it's essential to perform some data cleaning to ensure the data is properly formatted. Here are the key cleaning steps:

- **Removing White Spaces in Column Names**: BigQuery tables cannot have column names with white spaces. Therefore, we replace any white spaces in the column names with underscores.
- **Converting Date Columns**: We convert the `DATE_ID` column to a datetime format so that it's treated as a date in BigQuery.
- **Converting Numeric Columns**: Certain columns that are supposed to be numerical might have non-numeric values. We convert these columns to numeric format, replacing any non-numeric values with `NaN` (Not-a-Number).

In [4]:
df.columns = df.columns.str.replace(" ", "_").str.replace("[()]", "", regex=True).str.replace("-", "_")

df["DATE_ID"] = pd.to_datetime(df["DATE_ID"])

for col in df.columns:
    if col not in ["AREA", "DATE_ID", "ERBS", "EUTRANCELLFDD"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

df.head()

,AREA,DATE_ID,ERBS,EUTRANCELLFDD,LTE_CSFB_SR,RRC_Setup_Success_Rate_Service,ERAB_Setup_Success_Rate_All,Session_Setup_Success_Rate,Session_Abnormal_Release,Intra_Frequency_Handover_Out_Success_Rate,Radio_Network_Availability_Rate,Cell_Downlink_Average_Throughput,Cell_Uplink_Average_Throughput,DL_PDCP_User_Throughput,User_Uplink_Average_Throughput,DL_Resource_Block_Utilizing_Rate,UL_Resource_Block_Utilizing_Rate,Downlink_Traffic_Volume,Uplink_Traffic_Volume,Total_Traffic_Volume,Maximum_User_Number_RrcConn,pm_count,Max_DL_Cell_Downlink_Throughput,Max_UL_Cell_Downlink_Throughput,inter_freq_HO,UL_RSSI_dbm,SE_2,Average_CQI_nonHOME,volte_traffic_erl,volte_payload_DL,volte_payload_UL,pmCellDownTimeAuto,pmCellDownTimeMan,L_LATENCY_DL_ms,Integrity_PacketLossRate_DL,Integrity_PacketLossRate_UL
0,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002ML1,0.0,99.99393,99.98195,99.97589,0.06679,99.66047,99.97338,19789.33210,2677.57289,36897.11111,8292.20991,17.656,10.541,56468.78425,5908.32513,62377.10938,51,96,147057,40975,99.54023,-117.74849,2.59416,11.543,0.54444,0.00426,0.00255,23,0,3.14533,0.00322,0.01026
1,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002ML2,0.0,99.88980,99.91135,99.80125,0.09515,99.41480,99.98380,18049.37516,1739.05075,20333.29218,1032.97104,33.239,17.802,97340.72600,9219.08388,106559.80987,83,96,177330,31804,98.71007,-113.52638,1.88985,10.274,1.47361,0.01188,0.00950,14,0,3.54175,0.00866,0.07735
2,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002ML3,0.0,99.97177,99.93837,99.91015,0.06221,99.11193,99.98148,20524.85892,2059.30252,30578.76421,1609.04242,18.405,10.784,48805.99038,4460.34025,53266.33063,41,96,151519,47438,98.73418,-114.56841,2.11394,10.925,0.18611,0.00181,0.00121,16,0,3.38281,0.00714,0.37989
3,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002MT1,0.0,99.57377,99.46474,99.04079,0.24718,99.87531,99.98727,11494.63908,2067.32164,16281.88406,3348.21321,19.140,11.401,24817.88875,3556.79400,28374.68275,24,96,107227,25911,98.60075,-119.60126,2.04409,11.349,0.34167,0.00284,0.00172,11,0,3.44817,0.03223,0.01450
4,BPP,2023-05-01,BPP002MM1_GUNUNG_GUNTUR,BPP002MT2,0.0,99.24931,99.09149,98.34761,0.44175,99.49495,99.98843,10570.76424,1568.23771,10733.56481,1379.71641,31.603,11.964,39139.74400,4135.48675,43275.23075,31,96,57049,15136,98.67769,-113.31840,1.60831,10.997,0.18333,0.00147,0.00126,10,0,3.87995,0.02036,0.02991


In [5]:
df_hourly.columns = df_hourly.columns.str.replace(" ", "_").str.replace("[()]", "", regex=True).str.replace("-", "_")

df_hourly["DATE_ID"] = pd.to_datetime(df_hourly["DATE_ID"])

for col in df_hourly.columns:
    if col not in ["AREA", "DATE_ID", "ERBS", "EUTRANCELLFDD"]:
        df_hourly[col] = pd.to_numeric(df_hourly[col], errors="coerce")

df_hourly.head()

,AREA,DATE_ID,HOUR_ID,ERBS,EUTRANCELLFDD,LTE_CSFB_SR,RRC_Setup_Success_Rate_Service,ERAB_Setup_Success_Rate_All,Session_Setup_Success_Rate,Session_Abnormal_Release,Intra_Frequency_Handover_Out_Success_Rate,Radio_Network_Availability_Rate,Cell_Downlink_Average_Throughput,Cell_Uplink_Average_Throughput,DL_PDCP_User_Throughput,User_Uplink_Average_Throughput,DL_Resource_Block_Utilizing_Rate,UL_Resource_Block_Utilizing_Rate,Downlink_Traffic_Volume,Uplink_Traffic_Volume,Total_Traffic_Volume,Max_DL_Cell_Downlink_Throughput,Max_UL_Cell_Downlink_Throughput,L_LATENCY_DL_ms,Integrity_PacketLossRate_DL,Integrity_PacketLossRate_UL,inter_freq_HO,UL_RSSI_dbm,SE_2,Average_CQI_nonHOME,volte_traffic_erl,volte_payload_DL,volte_payload_UL,Band_,Sector_,NEID_,SiteID_
0,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108ME1,0,100.0000,100.00000,100.00000,0.0,100.0,100.0,18.452752,1.027181,36.291791,0.500244,13.833,15.000,1.526871,0.097257,1.624129,57.013,3.057,4.45460,0.00244,0.06323,100.0,-111.24809,2.58517,11.558,0.0,0.0,0.0,NaN,1,NaN,NaN
1,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108ME2,0,100.0000,100.00000,100.00000,0.0,100.0,100.0,17.245951,1.687589,44.481812,1.115079,18.833,22.667,3.489945,0.263139,3.753084,83.282,3.058,2.78735,0.00270,0.06924,100.0,-113.47277,3.87550,12.635,0.0,0.0,0.0,NaN,2,NaN,NaN
2,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108ME3,0,0.0000,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000,0.000,0.00000,0.00000,0.00000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,3,NaN,NaN
3,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108MF1,0,99.9239,99.87849,99.80248,0.0,0.0,100.0,19.568129,0.632613,45.233267,0.285659,12.833,10.333,1.518476,0.039733,1.558209,57.273,1.005,5.45246,0.12481,1.84608,100.0,-109.93618,3.10492,12.473,0.0,0.0,0.0,NaN,1,NaN,NaN
4,BTG,2023-09-01,0,SAA108MM1_MT_LONGWHEA,SAA108MF2,0,100.0000,100.00000,100.00000,0.0,100.0,100.0,17.565332,1.701387,29.490436,1.012616,26.000,22.000,3.706544,0.251886,3.958430,70.168,2.405,3.44076,0.00505,0.00054,100.0,-113.14835,2.75921,12.306,0.0,0.0,0.0,NaN,2,NaN,NaN


In the code below, we define schemas for our data. These schemas specify the data types for each column, ensuring compatibility with BigQuery for storage and querying. The `dtype_to_schema` function maps column data types to corresponding BigQuery types, maintaining consistency and accuracy in our database.

In [6]:
def dtype_to_schema(dtype):
    if dtype == "object":
        return "STRING"
    elif dtype == "int64":
        return "INT64"
    elif dtype == "float64":
        return "FLOAT64"
    elif "datetime" in str(dtype):
        return "DATE"
    else:
        return "UNKNOWN"

schema = [{"name": col, "type": dtype_to_schema(df[col].dtype)} for col in df.columns]
schema

[{'name': 'AREA', 'type': 'STRING'},
 {'name': 'DATE_ID', 'type': 'DATE'},
 {'name': 'ERBS', 'type': 'STRING'},
 {'name': 'EUTRANCELLFDD', 'type': 'STRING'},
 {'name': 'LTE_CSFB_SR', 'type': 'FLOAT64'},
 {'name': 'RRC_Setup_Success_Rate_Service', 'type': 'FLOAT64'},
 {'name': 'ERAB_Setup_Success_Rate_All', 'type': 'FLOAT64'},
 {'name': 'Session_Setup_Success_Rate', 'type': 'FLOAT64'},
 {'name': 'Session_Abnormal_Release', 'type': 'FLOAT64'},
 {'name': 'Intra_Frequency_Handover_Out_Success_Rate', 'type': 'FLOAT64'},
 {'name': 'Radio_Network_Availability_Rate', 'type': 'FLOAT64'},
 {'name': 'Cell_Downlink_Average_Throughput', 'type': 'FLOAT64'},
 {'name': 'Cell_Uplink_Average_Throughput', 'type': 'FLOAT64'},
 {'name': 'DL_PDCP_User_Throughput', 'type': 'FLOAT64'},
 {'name': 'User_Uplink_Average_Throughput', 'type': 'FLOAT64'},
 {'name': 'DL_Resource_Block_Utilizing_Rate', 'type': 'FLOAT64'},
 {'name': 'UL_Resource_Block_Utilizing_Rate', 'type': 'FLOAT64'},
 {'name': 'Downlink_Traffic_Vol

In [7]:
def dtype_to_schema(dtype):
    if dtype == "object":
        return "STRING"
    elif dtype == "int64":
        return "INT64"
    elif dtype == "float64":
        return "FLOAT64"
    elif "datetime" in str(dtype):
        return "DATE"
    else:
        return "UNKNOWN"

schema_hourly = [{"name": col, "type": dtype_to_schema(df_hourly[col].dtype)} for col in df_hourly.columns]
schema_hourly

[{'name': 'AREA', 'type': 'STRING'},
 {'name': 'DATE_ID', 'type': 'DATE'},
 {'name': 'HOUR_ID', 'type': 'INT64'},
 {'name': 'ERBS', 'type': 'STRING'},
 {'name': 'EUTRANCELLFDD', 'type': 'STRING'},
 {'name': 'LTE_CSFB_SR', 'type': 'INT64'},
 {'name': 'RRC_Setup_Success_Rate_Service', 'type': 'FLOAT64'},
 {'name': 'ERAB_Setup_Success_Rate_All', 'type': 'FLOAT64'},
 {'name': 'Session_Setup_Success_Rate', 'type': 'FLOAT64'},
 {'name': 'Session_Abnormal_Release', 'type': 'FLOAT64'},
 {'name': 'Intra_Frequency_Handover_Out_Success_Rate', 'type': 'FLOAT64'},
 {'name': 'Radio_Network_Availability_Rate', 'type': 'FLOAT64'},
 {'name': 'Cell_Downlink_Average_Throughput', 'type': 'FLOAT64'},
 {'name': 'Cell_Uplink_Average_Throughput', 'type': 'FLOAT64'},
 {'name': 'DL_PDCP_User_Throughput', 'type': 'FLOAT64'},
 {'name': 'User_Uplink_Average_Throughput', 'type': 'FLOAT64'},
 {'name': 'DL_Resource_Block_Utilizing_Rate', 'type': 'FLOAT64'},
 {'name': 'UL_Resource_Block_Utilizing_Rate', 'type': 'FLOAT

In the code below, we handle the decryption of credentials necessary for connecting to BigQuery from Python. It's important to note that the following two files, `encryption_key.key` and `encrypted_credentials.enc,` must be located in the same directory as this Jupyter Notebook for the decryption process to work.

- The `encryption_key.key` file contains the encryption key required for decryption.
- The `encrypted_credentials.enc` file stores the encrypted credentials.

In [8]:
with open("encryption_key_bigquery.key", "rb") as key_file:
    key = key_file.read()

cipher = Fernet(key)

with open("encrypted_credentials_bigquery.enc", "rb") as encrypted_file:
    encrypted_data = encrypted_file.read()

decrypted_data = cipher.decrypt(encrypted_data)

The following code is responsible for storing the prepared data in BigQuery. Notably, the crucial parameter here is `if_exists`. 

- Set `if_exists="replace"` if you intend to replace any existing data in the BigQuery table with the new data.
- Set `if_exists="append"` if you want to add the data to the existing table without replacing any records.

Please adjust the `if_exists` parameter according to your specific needs and preferences to control how the data is stored in the BigQuery dataset.

In [9]:
target_table = "monitoring_396408.tsel_nms"
target_table2 = "monitoring_396408.tsel_nms_hourly"
project_id = "monitoring-396408"
credentials = Credentials.from_service_account_info(eval(decrypted_data.decode()))
job_location = "asia-southeast2"

In [10]:
df.to_gbq(
    destination_table=target_table,
    project_id=project_id,
    if_exists="replace",
    location=job_location,
    chunksize=10_000,
    progress_bar=True,
    credentials=credentials,
    table_schema=schema
)

100%|██████████| 1/1 [00:00<00:00, 1001.98it/s]


In [11]:
df_hourly.to_gbq(
    destination_table=target_table2,
    project_id=project_id,
    if_exists="replace",
    location=job_location,
    chunksize=10_000,
    progress_bar=True,
    credentials=credentials,
    table_schema=schema_hourly
)

100%|██████████| 1/1 [00:00<?, ?it/s]


# Creating an Account with Firebase

For creating an account with Firebase, please check the explanation [video here](https://www.awesomescreenshot.com/video/21514061?key=3e454df93de759ed13b5cc41eb6fbeff)